In [508]:
import urllib.request  # импортируем модуль
import json
import re
import sqlite3
import os
from pymystem3 import Mystem
from datetime import datetime, date

In [509]:
def getting_Posts():  # достаем посты 
    con = sqlite3.connect("posts.db") # создаем базу данных для постов
    c = con.cursor()
    c.execute("""CREATE TABLE IF NOT EXISTS alldata(post_id, text_post, lenght, time, week)""")
    # передаем токен и адрес группы
    token = '8a8e5ba78a8e5ba78a8e5ba7958ae46ec088a8e8a8e5ba7d64fd62ef62f996b3344a884'
    group_id = '-148881515'
    offsets = [0, 100]
    posts = {}
    
    for off in offsets:
        req = urllib.request.Request('https://api.vk.com/method/wall.get?owner_id=%s&offset=%s&count=4&v=5.92&access_token=%s'% (group_id, off, token))  
        response = urllib.request.urlopen(req) 
        result = response.read().decode('utf-8')
        data = json.loads(result) 
        #data_f = data['response']['items']
      #  adding_info = data['response']['items']
        for i in data['response']['items']:
           # print(i, '\n\n')
            posts[i['id']] = [i['text'], i['date']]
        #posts = data['response']['items'].join()
    for key, value in posts.items():
        post_id = key
        text = value[0]
        date_time = value[1]
        print(post_id, '\n------\n', text, '\n------\n', date_time)
  #  print(posts)
        unixtime = date_time # извлекаем из информации о посте timestamp
        utc = datetime.fromtimestamp(unixtime) # конвертируем unixtime в utc
        week = datetime.isoweekday(utc) # достаем день недели пубоикации
        print('day', week)
        date = str(utc)
        print(date)
        hours = date.split() # достаем час публикации
      #  print(hours)
        hour = hours[1].split(":")
        c_hour = int(hour[0])
        #print(hour[0], type(hour[0]), type(c_hour), c_hour)
        if c_hour >= 16: # присваиваем категорию часам
            time = "3"
        if c_hour >= 8 and c_hour < 16:
            time = "2"
        if c_hour < 8:
            time = "1"
        print(time)
        
        clean_t = re.sub(r'[а-яА-ЯёЁ](-)[а-яА-ЯёЁ]', '', text)
        count_clean = re.findall(r'[а-яА-ЯёЁ]+', clean_t)
      #  print(clean_t, count_clean, len(count_clean), '\n\n')
        
        c.execute("INSERT INTO alldata VALUES (?, ?, ?, ?, ?)", (post_id, text, len(count_clean), time, week))
    con.commit()
    con.close()  # закрываем БД
    return posts

In [510]:
def getting_Id():
    posts_id = []
    con = sqlite3.connect("posts.db") # открываем базу данных
    c = con.cursor()
    for row in c.execute("SELECT post_id, text_post FROM alldata"):
        #print(row[0])
        posts_id.append(row[0])
    print(posts_id)
    con.commit()
    con.close()
    return posts_id

In [511]:
def comments(post_id):
    comments = {}
    con = sqlite3.connect("posts.db") # открываем базу данных
    c = con.cursor()
    c.execute("""CREATE TABLE IF NOT EXISTS comments(post_id, text_comment, user_id, city, age, sex, first_name, last_name)""")
    token = '8a8e5ba78a8e5ba78a8e5ba7958ae46ec088a8e8a8e5ba7d64fd62ef62f996b3344a884'
    group_id = '-148881515'
    offsets = [0, 50]
    today = date.today()
    print(today, '\n\n\n\n\-------------------\n\n\n\n\n\n')
    for i in post_id:
        for off in offsets:
            req = urllib.request.Request('https://api.vk.com/method/wall.getComments?owner_id=%s&post_id=%s&offset=%s&count=4&v=5.92&access_token=%s'%(group_id, i, off, token))
            try:
                response = urllib.request.urlopen(req)
                result = response.read().decode('utf-8')
                data = json.loads(result)
                for i in data['response']['items']:
                    print(i, '\n\n')
                    if 'from_id' in i and 'text' in i:
                        comments[i['id']] = [i['post_id'], i['text'], i['from_id']]
            except UnicodeEncodeError:
                pass
    for key, value in comments.items():
        comment_id = key
        text = value[1]
        post_item = value[0]
        user_id = value[2]
        print(user_id, text)
        req_2 = urllib.request.Request('https://api.vk.com/method/users.get?v=5.92&access_token=%s&user_ids=%s&fields=sex,bdate,city'  % (token, user_id))
        response_2 = urllib.request.urlopen(req_2)
        result_2 = response_2.read().decode('utf-8')
        data_2 = json.loads(result_2)
        #for i in data_2:
        print(data_2)
        for i in data_2['response']:
            print(i['id'])
            if 'sex' in i:
                sex = i['sex']
                if sex == 1:
                    sex = 'female'
                if sex == 2:
                    sex = 'male'
                if sex == 0:
                    sex = "None"
            else:
                sex = "None"
            if 'bdate' in i:
                bdate_start = i['bdate']
                bdate_list = bdate_start.split('.')
                print(bdate_list)
                if len(bdate_list) == 3:
                    bdate_str = str(bdate_list[2]) + '-' + str(bdate_list[1]) + '-' + str(bdate_list[0])
                    bdate_fi = datetime.strptime(str(bdate_str), "%Y-%m-%d")
                    bdate = today.year - bdate_fi.year - ((today.month, today.day) < (bdate_fi.month, bdate_fi.day))
                else:
                    bdate = "None"
            else:
                bdate = "None"
            if 'city' in i:
                city = i['city']['title']
            else:
                city = 'None'
            print(city)
            if 'first_name' in i:
                first_name = i['first_name']
            else:
                first_name = 'None'
            if 'last_name' in i:
                last_name = i['last_name']
            else:
                last_name = 'None'
        c.execute("INSERT INTO comments VALUES (?, ?, ?, ?, ?, ?, ?, ?)", (post_item, text, user_id, city, bdate, sex, first_name, last_name))
    print(len(comments))
    con.commit()
    con.close()
    return comments

In [512]:
#def length_Poco(comments, posts):
#    for i in comments:
#        print(i)
#    return

In [513]:
def main():
    take_posts = getting_Posts()
    for_comment = getting_Id()
    comment = comments(for_comment)
 #   length = length_Poco(take_posts, comment)
main()

7096090 
------
 C гoдaми люди cтaнoвятся мудpee👆🏻 
------
 1556346300
day 6
2019-04-27 09:25:00
2
7140591 
------
 15 причин никогда не жениться

1. Жениться всегда было принято исключительно на девственнице. Муж всегда обладал правом быть первым и единственным. Сейчас 99% женщин - это порванки, которых имел как хотел до вас кто-то другой. Брать в жены порванку, это все-равно что покупать подержанный автомобиль по цене нового, да еще и в кредит (и часто с прицепом в комплекте).

2. Вы не только будете не первым, но и далеко не единственным. Если раньше за измену женщину могли казнить или предать всеобщему позору, то сейчас никакого наказания за измену не предусмотрено, причем вершить самосуд нельзя по причине уголовного наказания, а среди самих женщин стало модно и популярно заводить любовников.

3. Более половины браков заканчиваются разводом с последующими алиментами. Это сухая и реальная статистика, с которой бессмысленно спорить. Вы же не думаете, что вы один такой самый умный и н

{'id': 7140611, 'from_id': 411067279, 'post_id': 7140591, 'owner_id': -148881515, 'parents_stack': [], 'date': 1556837896, 'text': 'Какая дичь', 'thread': {'count': 0, 'items': [], 'can_post': True, 'show_reply_button': True, 'groups_can_post': True}} 


{'id': 7140624, 'from_id': 95517200, 'post_id': 7140591, 'owner_id': -148881515, 'parents_stack': [], 'date': 1556837956, 'text': 'Чисто пост женоненавистника. Для меня этот пост как провокация.', 'thread': {'count': 0, 'items': [], 'can_post': True, 'show_reply_button': True, 'groups_can_post': True}} 


{'id': 7140628, 'from_id': 17222025, 'post_id': 7140591, 'owner_id': -148881515, 'parents_stack': [], 'date': 1556837987, 'text': 'Парень ты что из детдома?', 'thread': {'count': 0, 'items': [], 'can_post': True, 'show_reply_button': True, 'groups_can_post': True}} 


{'id': 7140629, 'from_id': 1059478, 'post_id': 7140591, 'owner_id': -148881515, 'parents_stack': [], 'date': 1556837987, 'text': 'Ты сука Еблан Ебланович Ебланов.', 'thr

{'response': [{'id': 211463484, 'first_name': 'Elena', 'last_name': 'Trofimova', 'is_closed': True, 'can_access_closed': False, 'sex': 1, 'bdate': '14.2', 'city': {'id': 2, 'title': 'Saint Petersburg'}}]}
211463484
['14', '2']
Saint Petersburg
177415556 Если в 20 этого не было, то далеко не факт, что к 30 – 40 появится😂
{'response': [{'id': 177415556, 'first_name': 'Irina', 'last_name': 'Martynova', 'is_closed': False, 'can_access_closed': True, 'sex': 1, 'bdate': '17.5', 'city': {'id': 72, 'title': 'Krasnodar'}}]}
177415556
['17', '5']
Krasnodar
11861192 Иллюстрация "Превращение в подкаблучника". На самом деле всё норм, не бугуртите.
{'response': [{'id': 11861192, 'first_name': 'Dmitry', 'last_name': 'Bobr', 'is_closed': False, 'can_access_closed': True, 'sex': 2, 'city': {'id': 1, 'title': 'Moscow'}}]}
11861192
Moscow
411067279 Какая дичь
{'response': [{'id': 411067279, 'first_name': 'Ekaterina', 'last_name': 'Nova', 'is_closed': False, 'can_access_closed': True, 'sex': 1, 'city': {'

{'response': [{'id': 382362619, 'first_name': 'Yulia', 'last_name': 'Sinkova', 'is_closed': False, 'can_access_closed': True, 'sex': 1, 'city': {'id': 73, 'title': 'Krasnoyarsk'}}]}
382362619
Krasnoyarsk
66390332 А кроме как рожать ума нет? Учиться пойти, работать, уехать с деревни..
{'response': [{'id': 66390332, 'first_name': 'Natalya', 'last_name': 'Startseva', 'is_closed': True, 'can_access_closed': False, 'sex': 1, 'bdate': '18.6', 'city': {'id': 97, 'title': 'Novokuznetsk'}}]}
66390332
['18', '6']
Novokuznetsk
264416970 Харе плодится, а так делай, че хочешь, хуже уже не будет.
{'response': [{'id': 264416970, 'first_name': 'Ksenia', 'last_name': 'Ivanova', 'is_closed': False, 'can_access_closed': True, 'sex': 1, 'bdate': '13.7', 'city': {'id': 2023, 'title': 'Tynda'}}]}
264416970
['13', '7']
Tynda
32


In [472]:
def make_Corpora1():
    con = sqlite3.connect("posts.db") # открываем базу данных
    c = con.cursor()
    for row in c.execute("SELECT * FROM alldata"):
        #print(row[0], row[1])
       # text_post = row[1]
        filename = "post_" + str(row[0]) + '.txt' # создает название файла
        dirname = "plain_postsVK" # создает название папки
        # прописвает пути
        path_dirname = "%s" % dirname
        if not os.path.exists(path_dirname):
            os.makedirs(path_dirname)
        # прописывает путь к файлу
        filepath = os.path.join(path_dirname, filename).encode('utf-8')
        # создает файл
       # c.execute('SELECT text_post, text_comment FROM alldata JOIN comments ON alldata.post_id = comments.post_id')
       # post_comment = c.fetchall()
       # print(post_comment)
       # for i in post_comment:
       #     print(i)
       #     if i[0] == text_post:
       #         print(i[1])
       #         comments_string = ''
       #         comments_string += i[1] + '\n'
        with open(filepath, 'w', encoding="utf-8") as f:
            texts = "@post\n" + row[1] + '\n@comments\n'
            f.write(texts)
    con.commit()
    con.close()
    
make_Corpora1()

In [501]:
def make_Corpora2():
   # con = sqlite3.connect("posts.db") # открываем базу данных
   # c = con.cursor()
    start_path = './plain_postsVK/'
    for root, dirs, files in os.walk(start_path):
        for file in files:
            if ".txt" in file:
                final = os.path.join(root, file)
                name_file = re.search(r'post_(.*).txt', file, re.DOTALL)
                post_id = (int(name_file.group(1)),)
                print(post_id)
                con = sqlite3.connect("posts.db") # открываем базу данных
                c = con.cursor()
                c.execute('SELECT post_id, text_comment FROM comments WHERE post_id=?', post_id)
                #print(c.fetchall())
                post_comment = c.fetchall()
                print(post_comment)
                con.close()
   # con.commit()
  #  con.close()
    
make_Corpora2()

(7098414,)
[(7098414, 'Вот кому делать нехер...'), (7098414, 'Жена ,под конец жизни ,из одежды выросла😕'), (7098414, 'И только на двух  фотографиях жена улыбается😆...'), (7098414, 'Жена блин вааще огонь🤣'), (7098414, 'За 25 лет не заработали на утюг?! Че они мятые такие?'), (7098414, 'Депрессивные снимки, посмотришь жить не хочется. Фотограф унылое го.вно'), (7098414, 'Мама всю жизнь была третьим мальчиком в семье'), (7098414, 'Меня терзают смутные сомнения.... А не постановка ли это?'), (7098414, 'Грустно,на такую семью смотреть.'), (7098414, 'Печальненькая семья.Только они и согласились 25 лет ждать фотографа.')]
(7140024,)
[(7140024, 'Еще одна любительница таджиков. Это что, эпидемия какая то?🤦\u200d♂️'), (7140024, ''), (7140024, 'Чернильница😀'), (7140024, 'никаких алимеетов и не будет'), (7140024, 'О тоже послушаю ,вдруг пригодиться !'), (7140024, 'Мде... и что вы в нем нашли? Волосатую спину?)'), (7140024, 'Фу, чернильница мерзкая'), (7140024, 'бляяя'), (7140024, 'Ну че в России с